<a href="https://colab.research.google.com/github/antoniomedina99/Curso_NPL/blob/main/Notebooks/Clasificaci%C3%B3n_multiclase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <Center> Redes neuronales para clasificación del dataset iris

Importar librerías

In [2]:
# Manejo de datos y graficas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Creacion de modelos
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from scikeras.wrappers import KerasClassifier
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder

import warnings
# Suprimir warnings (las redes neuronales arojan mucho warnings es mejor suprimir para mejor presentacion)
warnings.simplefilter("ignore")

# 1. cargar dataframe

In [6]:
dataframe = pd.read_csv("https://drive.google.com/uc?id=1lF-WSgmqNzS2MZobzNYZA12uhCWyxoYr",
                        header=None)
''' El header=None es para indicar que las columnas no tienen nombre'''
# Mostrar data
dataframe

,0,1,2,3,4
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


In [18]:
# Separar variables predictoras
dataset = dataframe.values
''' Aquí se transforma el objeto pandas en un array de numpy'''
dataframe.head(3) # Mostraos las 3 primeras filas

,0,1,2,3,4
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa


In [19]:
# Extraer X y y
X = dataset[:,0:4].astype(float)  # extraer las primeras 4 variables del data frame
y = dataset[:,4] # Aquí extraemos la última variable

# 2. Codificar las variables de salida

la variable de salida tiene 3 valores: Setosa, vesocolor y virginica.

Metodo:

1. Codificar las cadenas de manera coherente en números enteros utilizando la clase `LabelEcoder`de scikit-lear.
2. Luego, convertimos el vector de numeros enteros en One_hote unsando la función de keras `to_categorical()`.

In [20]:
ecoder = LabelEncoder() # Crear el objeto LabelEncoder.
ecoder.fit(y) #Ajustar el encoder a los valores de Y (entrenar el encoder).
encoded_y = ecoder.transform(y) #Transformar los valores categóricos en números.

In [21]:
#  Setosa = 0, vesocolor = 1 y virginica = 2.
encoded_y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [22]:
# Ver las clases asignadas
print(ecoder.classes_)

['Iris-setosa' 'Iris-versicolor' 'Iris-virginica']


In [23]:
# Aplicar One-Hot Encoding
dummy = to_categorical(encoded_y)


In [24]:
# Mostrar los primeros 5 datos
# El one hot se define teniendo en cuenta la configuración del ecoder
# Iris-setosa = [1,0,0] / Iris-versicolor= [0,1,0] / Iris-virginica= [0,0,1]
dummy[:5]

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.]])

# 3 Definir la red neuronal
A continuación se muestra el procedimiento a la hora de crear la función a trabajar:
1. Crea una red simple completamente conectada con una capa oculta de 8 neuronas. 
2. La capa oculta utiliza una función de activación ReLu. 
3. Debido a que utilizamos One-Hot Encoding, la capa de salida debe crear 3 valores de salida, uno para cada clase. 
4. El valor de salida con el valor más grande se tomará como la clase predicha por el modelo. La topología quedaría así:
```
    4 inputs -> [8 hidden nodes] -> 3 outputs
```
5. Tendremos una función de activación Softmax en la capa de salida. 
6. Finalmente, la red utiliza Adam con una función de pérdida logarítmica (`categorical_crossentropy`).

## Funciones de Pérdidas para Clasificación  

| Tipo de pérdida                  | Uso recomendado                                      | Fórmula |
|-----------------------------------|------------------------------------------------------|---------|
| **`binary_crossentropy`**         | Para clasificación binaria (2 clases)               | $ L = - \frac{1}{N} \sum [y \log(\hat{y}) + (1 - y) \log(1 - \hat{y})] $ |
| **`categorical_crossentropy`**    | Para clasificación multiclase (etiquetas en one-hot) | $ L = - \sum y_i \log(\hat{y}_i) $ |
| **`sparse_categorical_crossentropy`** | Como `categorical_crossentropy` pero para etiquetas enteras (no one-hot) | Similar a `categorical_crossentropy` |
| **`KLDivergence` (Kullback-Leibler Divergence)** | Para medir la diferencia entre distribuciones de probabilidad | $ L = \sum y_i \log \left(\frac{y_i}{\hat{y}_i} \right) $ |

In [25]:
# se define el base line
def baseline_model():
    model = Sequential()
    model.add(Dense(8, input_dim=4, activation= 'relu'))
    model.add(Dense(3, activation='softmax'))

    #Compilar red neuronal
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics= ['accuracy'])

    return model

In [26]:
# Ahora se crea el contenedor para utilizarlo en scikit-learn

estimator = KerasClassifier(build_fn= baseline_model, epochs=200, batch_size=5)

# Evalur modelo mediante croz validation con 10 folks

In [27]:
# Al objeto result se le pasa X o variables predictoras ya preparadas
# la variable y o donde están las etiquetas de las clases en formato one hot/categorica
# metodo de entrenamiento Kfold

# Crear el objeto KFold
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Evaluar el modelo con validación cruzada
results = cross_val_score(estimator, X, dummy, cv=kf, scoring='accuracy', verbose=0)

# Imprimir la media del accuracy y la desviación estándar
print(f'El accuracy es de: {results.mean() * 100:.2f}%, la desviación estándar es de: {np.std(results) * 100:.2f}%')

Epoch 1/200
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.2980 - loss: 1.4186   
Epoch 2/200
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5756 - loss: 1.0634 
Epoch 3/200
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5282 - loss: 0.9525 
Epoch 4/200
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6391 - loss: 0.7713
Epoch 5/200
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6507 - loss: 0.7282     
Epoch 6/200
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6894 - loss: 0.6626 
Epoch 7/200
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7755 - loss: 0.5897 
Epoch 8/200
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7285 - loss: 0.5484 
Epoch 9/200
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8319 - loss: 0.4788 
Epoch 10/200
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8500 - loss: 0.4531 
Epoch 11/200
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7903 - loss: 0.4695 
Epoch 12/200
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/